In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

INPUT_FILE = "gold_news_10years.csv"
OUTPUT_FILE = "gold_training_data_scored.csv"

# ================= 核心：评分逻辑 =================
# 我们根据收益率的分布来打分
# 比如：涨幅 > 2% 就是 +5 分
# 涨幅 0.2% 就是 +1 分
# 这样模型能学会"程度"

def get_score_from_return(ret):
    # 这里的阈值可以根据黄金的历史波动率调整
    # 黄金波动比个股小，一般日涨跌超过 1.5% 就算很大了
    r = ret * 100 # 转成百分比
    
    if r > 2: return 5
    elif r > 1.4: return 4
    elif r > 0.8: return 3
    elif r > 0.4: return 2
    elif r > 0.1: return 1
    elif r < -2: return -5
    elif r < -1.4: return -4
    elif r < -0.8: return -3
    elif r < -0.4: return -2
    elif r < -0.1: return -1
    else: return 0 # -0.1 ~ 0.1 算绝对中性

def process_labels():
    print("📈 下载黄金期货数据 (GC=F)...")
    # 下载范围涵盖新闻时间
    gold = yf.download("GC=F", start="2020-01-01", end="2026-01-01", progress=False)['Close']
    
    # 计算当日收益率 (Close - Open) / Open
    # 因为黄金是24小时，我们假设新闻是在当天交易时段内发生的，看当天的收盘反应
    # 或者简单点：(Close_t - Close_t-1) / Close_t-1
    returns = gold.pct_change()
    
    print("🔄 读取新闻并打分...")
    try:
        df_news = pd.read_csv(INPUT_FILE)
    except:
        print("❌ 找不到新闻文件")
        return

    scored_data = []
    
    for idx, row in df_news.iterrows():
        try:
            date_str = row['Date']
            dt = pd.to_datetime(date_str)
            
            # 找当天收益率
            if dt not in returns.index:
                continue # 如果当天没开盘（周末），这新闻先丢弃，或者归到周一（这里简单丢弃）
            
            ret = returns.loc[dt].item() # 获取数值
            
            score = get_score_from_return(ret)
            
            # 构建 Llama-3 训练数据格式
            # 我们直接让 Output 就是分数，方便解析
            scored_data.append({
                'Date': date_str,
                'Headline': row['Headline'],
                'Return': ret,
                'Score': score
            })
        except: continue
        
    df_final = pd.DataFrame(scored_data)
    df_final.to_csv(OUTPUT_FILE, index=False)
    
    print(f"✅ 打分完成！文件: {OUTPUT_FILE}")
    print("📊 分数分布:")
    print(df_final['Score'].value_counts().sort_index())
    
    # 画个图看看分布是否合理
    plt.figure(figsize=(10,5))
    df_final['Score'].hist(bins=11)
    plt.title("Gold News Sentiment Score Distribution (-5 to +5)")
    plt.savefig("score_dist.png")

if __name__ == "__main__":
    process_labels()